In [1]:
import bs4 as bs
import pandas as pd
import matplotlib as plt
from matplotlib import pyplot as pplt
import os
from pptx import Presentation

#Die Buch-Kürzel des alten Testaments
AT = ["Gn", "Ex", "Lv", "Nm", "Dt", "Ios", "Idc", "Rt", "1Sm", "2Sm", "3Rg", "4Rg", "1Par", "2Par", "Esr", "Neh", "Tb",
      "Idt", "Est", "1Mcc"
    , "2Mcc", "3Mcc", "4Mcc", "Iob", "Ps", "Prv", "Ecl", "Ct", "Sap", "Sir", "Is", "Ier", "Lam", "Bar", "Ez", "Dn",
      "Os", "Ioel", "Am", "Abd"
    , "Ion", "Mi", "Na", "Hab", "So", "Agg", "Za", "Mal"]


  
def percent_calc(num1, num2):
    '''returns percentage value'''
    return (num1/num2)*100
    
 
    
def analyse(filename):
    '''Opens a file specified by the filename, including the directory and creates dataframe for the respective file. Subsequently, this function analyses the chapter with regard to our criteria'''
    file_title = filename.split(".")[0]
    with open(filename, 'r') as f:
        file = f.read()   
    soup = bs.BeautifulSoup(file, 'lxml')
    Refs = soup.select('ref[cRef]')
    # Konvertierung der Daten in data. Dies ist die Grundlage für die Auswertungen
    data = []
    for ref in Refs:
        if ref.get('cref').split('_')[0] in AT:
            if ref.parent.find_previous_sibling() == ref.parent.find_previous_sibling('q'):
                data.append((ref.get('cref') , 'at', ref.getText() , ref.parent.find_previous_sibling('q').get_text(' ',strip=True)))
            else:
                data.append((ref.get('cref'), 'at', ref.getText(), 'not a direct quote'))
        else:
            if ref.parent.find_previous_sibling() == ref.parent.find_previous_sibling('q'):
                data.append((ref.get('cref') , 'nt', ref.getText() , ref.parent.find_previous_sibling('q').get_text(' ',strip=True)))
            else:
                data.append((ref.get('cref'), 'nt', ref.getText(), 'not a direct quote'))
    #print(data)
    # der eigentliche Dataframe wird gesetzt
    df_data = pd.DataFrame(data)
    # der Header wird gesetzt.
    df_data.columns = ['kuerzel', 'atnt', 'stelle', 'zitat']



    # Hier wird eine Liste der wörtlichen Zitate erstellt
    qs = []
    for ref in Refs:
        if ref.parent.find_previous_sibling() == ref.parent.find_previous_sibling("q"):
            qs.append(ref.parent.find_previous_sibling("q").get_text(" ", strip=True))



    stellen = []
    for i in range (0, len(df_data)):
        if df_data['zitat'][i] != 'not a direct quote':
            stellen.append(df_data['stelle'][i])
    
    # Zählen der Gesamtlänge des zitierten Textes; one Leerzeilen und Zeilenumbrüche 
    quote_len = 0
    for i in range(len(qs)):
        quote_len +=len(qs[i].replace(" ","").replace("\n", ""))
        i+=1



    ges = bs.BeautifulSoup(file, 'lxml')

    ges_soup = ges.select('div')
    total_len=0
    for i in range(len(ges_soup)):
        total_len += len(ges_soup[i].getText().replace(" ", "").replace("n", ""))
    
    print(f"Anzahl an erfassten Bibelstellen: {len(Refs)}")
    print(f"wörtliche Zitate: {len(stellen)}")
    print(f"Liste d. wörtlichen Zitate:{stellen}")
    print("Gesamtlänge:" + str(total_len))
    print("Zitatlänge :"+str(quote_len))    
    print("Prozentualer Anteil: " +(str(percent_calc(quote_len, total_len))))
    lines = [f"Anzahl an erfassten Bibelstellen: {len(Refs)}", f"wörtliche Zitate: {len(stellen)}", f"Liste d. wörtl.: {stellen}", f"Gesamtlänge: {str(total_len)}", f"Zitatlänge :{str(quote_len)}",f"Prozentualer Anteil: {(str(percent_calc(quote_len, total_len)))}"]
    # Speichern der obigen Ausgabe harter Fakten in einer Text-Datei mit dem Titel der ausgewählten Datei. 
    with open(f'Visualisierungen/{file_title}', 'w') as f2:
        for line in lines:
            f2.write(line)
            f2.write("\n")
    # Ausgabe, Speichern und Schließen der einzelen Diagramme. 
    print(df_data["kuerzel"].value_counts().plot(figsize=(8,8), kind="pie", legend=False, title=f"{filename}"))
    pplt.savefig(f'Visualisierungen/{file_title}.jpg')
    pplt.clf()
    pplt.cla()
    pplt.close()
    print(df_data["atnt"].value_counts().plot(figsize=(8,8), kind="bar", legend=False, title=f"{filename}", color=["blue", "red"]))
    pplt.savefig(f'Visualisierungen/{file_title}_atnt.jpg')
    pplt.clf()
    pplt.cla()
    pplt.close()
    print(df_data["kuerzel"].value_counts()[:5].plot(figsize=(8,8), kind="barh", legend=False, title=f"{filename}  @cRef)", color="purple"))
    pplt.savefig(f'Visualisierungen/{file_title}_cref.jpg')
    pplt.clf()
    pplt.cla()
    pplt.close()
    print(df_data["stelle"].value_counts()[:10].plot(figsize=(8,8), kind="barh", legend=False, title=f"{filename} nach Stellen", color="red"))
    pplt.savefig(f'Visualisierungen/{file_title}_nach_stellen.jpg')
    pplt.clf()
    pplt.cla()
    pplt.close()
    
    

# Daten für das gesamte Interim

In [2]:
#Probe-Sachen
with open(f'interim.xml', 'r') as f:
    file = f.read()
soup = bs.BeautifulSoup(file, 'lxml')
Refs = soup.select('ref[cRef]')

# Eine leere Liste wird erstellt und mit den Bibelstellen gefüllt. Hierbei wird sortiert, ob sie im Alten Testament stehen oder nicht.
# Ein zweites Auswahlkriterium ist die Frage, ob direkt vor dem Eltern-ELement des <ref type='biblical'> ein <q> steht, und somit ein direktes Zitat vorliegt.
data = []
for ref in Refs:
    if ref.get('cref').split('_')[0] in AT:
        if ref.parent.find_previous_sibling() == ref.parent.find_previous_sibling('q'):
            data.append((ref.get('cref') , 'at', ref.getText() , ref.parent.find_previous_sibling('q').get_text(' ',strip=True)))
        else:
            data.append((ref.get('cref'), 'at', ref.getText(), 'not a direct quote'))
    else:
        if ref.parent.find_previous_sibling() == ref.parent.find_previous_sibling('q'):
            data.append((ref.get('cref') , 'nt', ref.getText() , ref.parent.find_previous_sibling('q').get_text(' ',strip=True)))
        else:
            data.append((ref.get('cref'), 'nt', ref.getText(), 'not a direct quote'))
# der eigentliche Dataframe wird gesetzt
df_data = pd.DataFrame(data)
# der Header wird gesetzt.
df_data.columns = ['kuerzel', 'atnt', 'stelle', 'zitat']



stellen = []
for i in range (0, len(df_data)):
    if df_data['zitat'][i] != 'not a direct quote':
        stellen.append(df_data['stelle'][i])
print(len(stellen))

# Hier wird eine Liste der wörtlichen Zitate erstellt
qs = []
for ref in Refs:
    if ref.parent.find_previous_sibling() == ref.parent.find_previous_sibling("q"):
        qs.append(ref.parent.find_previous_sibling("q").get_text(" ", strip=True))


# Zählen der Gesamtlänge des zitierten Textes; one Leerzeilen und Zeilenumbrüche
quote_len = 0
for i in range(len(qs)):
    quote_len +=len(qs[i].replace(" ","").replace("\n", ""))
    i+=1



ges = bs.BeautifulSoup(file, 'lxml')

ges_soup = ges.select('div')
total_len=0
for i in range(len(ges_soup)):
    total_len += len(ges_soup[i].getText().replace(" ", "").replace("n", ""))

lines = [f"Anzahl an erfassten Bibelstellen: {len(Refs)}", f"wörtliche Zitate: {len(qs)}", f"Gesamtlänge: {str(total_len)}", f"Zitatlänge :{str(quote_len)}",f"Prozentualer Anteil: {(str(percent_calc(quote_len, total_len)))}"]
with open('Visualisierungen/interim.txt', 'w') as f2:
    for line in lines:
        f2.write(line)
        f2.write("\n")

print(f"Anzahl an erfassten Bibelstellen: {len(Refs)}")
print(f"wörtliche Zitate: {len(qs)}")
print("Gesamtlänge:" + str(total_len))
print("Zitatlänge :"+str(quote_len))
print("Prozentualer Anteil: " +(str(percent_calc(quote_len, total_len))))


print(df_data.groupby("atnt").count())

114
Anzahl an erfassten Bibelstellen: 283
wörtliche Zitate: 114
Gesamtlänge:204612
Zitatlänge :9713
Prozentualer Anteil: 4.747033409575196
      kuerzel  stelle  zitat
atnt                        
at         57      57     57
nt        226     226    226


In [3]:
analyse('interim_26/ch_02.xml')

Anzahl an erfassten Bibelstellen: 14
wörtliche Zitate: 1
Liste d. wörtlichen Zitate:['Röm 5,12']
Gesamtlänge:2749
Zitatlänge :75
Prozentualer Anteil: 2.728264823572208
AxesSubplot(0.1275,0.11;0.77x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)


In [15]:
test = os.listdir('interim_26')
prs = Presentation()
lyt = prs.slide_layouts[0]
slide = prs.slides.add_slide(lyt)
title = slide.shapes.title
subtitle = slide.placeholders[1]
title.text="Das Augsburger Interim"
subtitle.text="Auswertungen"

for el in test:
    lyt2 = prs.slide_layouts()
    slide = prs.slides.add_slide(lyt)
    title = slide.shapes.title
    title.text = el

    

prs.save('test.pptx')

In [17]:
test = os.listdir('interim_26')
print(test)

for el in test:
    print(el.split(".")[0])

['ch_07.xml', 'ch_17.xml', 'ch_24.xml', 'ch_04.xml', 'ch_06.xml', 'ch_26.xml', 'ch_02.xml', 'ch_08.xml', 'ch_19.xml', 'Vorrede.xml', 'ch_12.xml', 'ch_18.xml', 'ch_21.xml', 'ch_11.xml', 'ch_13.xml', 'ch_01.xml', 'ch_16.xml', 'ch_10.xml', 'Vorrede.txt', 'ch_05.xml', 'ch_20.xml', 'schluss.xml', 'ch_14.xml', 'ch_22.xml', 'ch_25.xml', 'ch_15.xml', 'ch_09.xml', 'ch_03.xml', 'ch_23.xml']
ch_07
ch_17
ch_24
ch_04
ch_06
ch_26
ch_02
ch_08
ch_19
Vorrede
ch_12
ch_18
ch_21
ch_11
ch_13
ch_01
ch_16
ch_10
Vorrede
ch_05
ch_20
schluss
ch_14
ch_22
ch_25
ch_15
ch_09
ch_03
ch_23
